### <div style="color:#ffffff; background-color:#666666; padding:15px; border-radius:25px; text-align:center;">Gated Recurrent Unit</div>

<b>Gated Recurrent Unit :</b> GRU (Gated Recurrent Unit) is a simplified variant of LSTM with fewer gates, making it computationally more efficient while still handling long-range dependencies effectively.

<div style="background-color:#ffffff">
<img src="https://blog.eson.org/images/raw/NN%20-%20RNN%20-%20GRU%20with%20Equation%20-%20colah.png">
</div>

###
<div style="color:#ffffff; background-color:#666666; padding:15px; border-radius:25px; ">


<p><b>Update Gate:
 </b>This gate controls how much of the previous hidden state should be passed to the next time step. It plays the role of both the forget and input gates in LSTMs.

</p>
<p><b> Reset Gate:
 </b>This gate decides how much of the previous hidden state should be "forgotten" or reset before updating the new hidden state.
 </p>
<p><b>Candidate Hidden State:
 </b>It is similar to the LSTM cell state's candidate vector, generated by the current input and the reset hidden state, using a tanh activation function.
</p>
<p><b>Hidden State Update:
</b>The hidden state is updated using the update gate to interpolate between the previous hidden state and the new candidate hidden state.
</p>


</div>

In [2]:
import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SpatialDropout1D,GRU
from sklearn.model_selection import train_test_split
import re

In [3]:
df = pd.read_csv("D:\Projects\Deep Learning\Deep-Learning\Sentiment.csv")
df.columns

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Parthiban\AppData\Local\Temp\ipykernel_9436\2645148268.py:1: SyntaxWarning: invalid escape sequence '\P'
  df = pd.read_csv("D:\Projects\Deep Learning\Deep-Learning\Sentiment.csv")


Index(['id', 'candidate', 'candidate_confidence', 'relevant_yn',
       'relevant_yn_confidence', 'sentiment', 'sentiment_confidence',
       'subject_matter', 'subject_matter_confidence', 'candidate_gold', 'name',
       'relevant_yn_gold', 'retweet_count', 'sentiment_gold',
       'subject_matter_gold', 'text', 'tweet_coord', 'tweet_created',
       'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [4]:
# Keeping only the neccessary columns
data = df[['text','sentiment']]


In [5]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Parthiban\AppData\Local\Temp\ipykernel_9436\3498670558.py:3: SyntaxWarning: invalid escape sequence '\s'
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
C:\Users\Parthiban\AppData\Local\Temp\ipykernel_9436\3498670558.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt',' ')
C:\Users\Parthiban\AppData\Local\Temp\ipykernel_9436\3498670558.py:9: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt',' ')


4472
16986


In [6]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(GRU(196, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

c:\Users\Parthiban\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [8]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
225/225 - 9s - 39ms/step - accuracy: 0.8111 - loss: 0.4460
Epoch 2/7
225/225 - 6s - 25ms/step - accuracy: 0.8574 - loss: 0.3371
Epoch 3/7
225/225 - 8s - 34ms/step - accuracy: 0.8798 - loss: 0.2881
Epoch 4/7
225/225 - 12s - 55ms/step - accuracy: 0.8904 - loss: 0.2590
Epoch 5/7
225/225 - 6s - 29ms/step - accuracy: 0.9046 - loss: 0.2390
Epoch 6/7
225/225 - 10s - 43ms/step - accuracy: 0.9122 - loss: 0.2159
Epoch 7/7
225/225 - 10s - 43ms/step - accuracy: 0.9243 - loss: 0.1925


In [9]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

64/64 - 2s - 33ms/step - accuracy: 0.8143 - loss: 0.4281
score: 0.43
acc: 0.81
